In [1]:
import numpy as np
import random
from sympy import symbols, Eq, sympify, lambdify

In [2]:
x, y, z, t = symbols('x y z t')

In [4]:
GENERATIONS = 10000
POPULATION_SIZE = 5000
MUTATION_RATE = 0.4
MUTATION_RANGE = 10000
PICK_NUMBER_FROM_PREV_GEN = 1000
PICK_NUMBER_FOR_PARENTS = 1000
NUMBER_OF_EQUATIONS = None
STAGNATION_LIMIT = 30
MAX_VALUE = 1e7
MIN_VALUE = -1e7

In [6]:
def initial_generation():
    return np.random.uniform(MIN_VALUE, MAX_VALUE, size=(POPULATION_SIZE, NUMBER_OF_EQUATIONS))


def parse_equation_string(equation_string):
    left_expression, right_value = equation_string.split("=")
    left_expr = sympify(left_expression.strip())
    right_expr = float(right_value.strip())
    return Eq(left_expr, right_expr)


def crossover(p1, p2):
    alpha = random.random()
    return alpha * p1 + (1 - alpha) * p2


def mutate(individual):
    if random.random() > MUTATION_RATE:
        return individual
    noise = np.random.normal(0, MUTATION_RANGE, size=NUMBER_OF_EQUATIONS)
    mutated = np.clip(individual + noise, MIN_VALUE, MAX_VALUE)
    return mutated


def select_parent(population, scores):
    indices = random.sample(range(PICK_NUMBER_FOR_PARENTS), 20)
    best_index = min(indices, key=lambda i: scores[i])
    return population[best_index]


def geneticAlgorithm(numberOfEquation, fitNessFunction, functions):
    population = initial_generation()
    stagnation_count = 0
    previous_best = None

    for generation in range(GENERATIONS):
        scores = np.array([fitNessFunction(index, functions)
                          for index in population])
        sorted_indices = np.argsort(scores)
        population = population[sorted_indices]
        scores = scores[sorted_indices]

        current_best = population[0]
        current_loss = scores[0]

        if current_loss <= 0.00001:
            print(f"🧬 Stopped after {generation + 1} generations")
            break
        if previous_best is not None and np.allclose(current_best, previous_best, rtol=1e-5, atol=1e-5):
            stagnation_count += 1
        else:
            stagnation_count = 0

        previous_best = current_best.copy()

        if stagnation_count >= STAGNATION_LIMIT:
            print(
                f"🛑 Stopped due to stagnation after {generation + 1} generations")
            break

        next_generation = population[:PICK_NUMBER_FROM_PREV_GEN].tolist()

        while len(next_generation) < POPULATION_SIZE:
            p1 = select_parent(population, scores)
            p2 = select_parent(population, scores)
            child = crossover(p1, p2)
            child = mutate(child)
            next_generation.append(child)

        population = np.array(next_generation)
        if (numberOfEquation == 2):
            print(
                f"Gen {generation+1}: x={current_best[0]:.4f}, y={current_best[1]:.4f} Loss={current_loss:.8f}")
        elif (numberOfEquation == 3):
            print(
                f"Gen {generation+1}: x={current_best[0]:.4f}, y={current_best[1]:.4f}, z={current_best[2]:.4f}, Loss={current_loss:.8f}")
        elif (numberOfEquation == 4):
            print(
                f"Gen {generation+1}: x={current_best[0]:.4f}, y={current_best[1]:.4f}, z={current_best[2]:.4f}, t={current_best[3]:.4f} Loss={current_loss:.8f}")
    return population[0]

In [8]:
# Q1

NUMBER_OF_EQUATIONS = 2


def q1fitness(ind, funcs):
    x_val, y_val = ind
    total_error = 0
    try:
        for f in funcs:
            val = f(x_val, y_val)
            total_error += abs(val)
    except Exception:
        return float('inf')
    return total_error


def q1_create_functions(parsed_eqs):
    funcs = []
    for eq in parsed_eqs:
        f = lambdify((x, y), eq.lhs - eq.rhs, 'numpy')
        funcs.append(f)
    return funcs


input_equations = []
for i in range(NUMBER_OF_EQUATIONS):
    equation = input(f"Enter equation {i+1} (e.g., ax+by=c): ")
    input_equations.append(equation)

parsed_eqs = [parse_equation_string(eq) for eq in input_equations]
funcs = q1_create_functions(parsed_eqs)

answer = geneticAlgorithm(2, functions=funcs, fitNessFunction=q1fitness)
final_loss = q1fitness(answer, funcs)

print("\n✅ Best solution found:")
print(f"x = {answer[0]:.2f}")
print(f"y = {answer[1]:.2f}")
print(f"🔍 Total error: {final_loss:.2f}")

Gen 1: x=378513.1349, y=-328964.8630 Loss=378513.13486884
Gen 2: x=51515.2996, y=-49110.1385 Loss=51515.29958711
Gen 3: x=-6631.3318, y=6826.9234 Loss=8186.06442766
Gen 4: x=-569.3554, y=561.8524 Loss=569.35538055
Gen 5: x=-106.7828, y=95.2503 Loss=106.78276224
Gen 6: x=8.3974, y=-8.1902 Loss=16.73946572
Gen 7: x=-1.7200, y=3.0141 Loss=2.07278934
Gen 8: x=0.1682, y=1.0870 Loss=0.16816458
Gen 9: x=-0.0003, y=1.2499 Loss=0.00277168
Gen 10: x=0.0007, y=1.2496 Loss=0.00143538
Gen 11: x=0.0001, y=1.2499 Loss=0.00034217
Gen 12: x=0.0000, y=1.2500 Loss=0.00002775
🧬 Stopped after 13 generations

✅ Best solution found:
x = -0.00
y = 1.25
🔍 Total error: 0.00


In [ ]:
# Q2
NUMBER_OF_EQUATIONS = 3


def q2fitness(ind, funcs):
    x_val, y_val, z_val = ind
    total_error = 0
    try:
        for f in funcs:
            val = f(x_val, y_val, z_val)
            total_error += abs(val)
    except Exception:
        return float('inf')
    return total_error


def q2_create_functions(parsed_eqs):
    funcs = []
    for eq in parsed_eqs:
        f = lambdify((x, y, z), eq.lhs - eq.rhs, 'numpy')
        funcs.append(f)
    return funcs


input_equations = []
for i in range(NUMBER_OF_EQUATIONS):
    equation = input(f"Enter equation {i+1} (e.g., 2*x + 3*y - z = 10): ")
    input_equations.append(equation)

parsed_eqs = [parse_equation_string(eq) for eq in input_equations]
funcs = q2_create_functions(parsed_eqs)

answer = geneticAlgorithm(3, functions=funcs, fitNessFunction=q2fitness)
final_loss = q2fitness(answer, funcs)

print("\n✅ Best solution found:")
print(f"x = {answer[0]:.2f}")
print(f"y = {answer[1]:.2f}")
print(f"z = {answer[2]:.2f}")
print(f"🔍 Total error: {final_loss:.3f}")

Gen 1: x=532884.8552, y=-5228392.7006, z=-2298.8305, Loss=9826802268.02929688
Gen 2: x=177072.4450, y=-3560332.1176, z=-138.0126, Loss=212588603.87121087
Gen 3: x=-20.6412, y=-25083.6897, z=38109.4733, Loss=6732785.74800524
Gen 4: x=130977.9353, y=519758.0476, z=-0.4908, Loss=1038807.16613603
Gen 5: x=188.0031, y=-56687.1973, z=108.2900, Loss=306563.25482570
Gen 6: x=-33.8496, y=1151.4544, z=-21.6711, Loss=11449.16112085
Gen 7: x=-25.4739, y=-1058.5823, z=-18.3480, Loss=6064.33766918
Gen 8: x=-6.1353, y=4.7213, z=-7.4851, Loss=514.96460346
Gen 9: x=0.2395, y=17.6796, z=-0.4799, Loss=79.04943618
Gen 10: x=4.5680, y=2.6491, z=0.0444, Loss=9.64725404
Gen 11: x=4.9029, y=4.4952, z=-0.1392, Loss=1.12343581
Gen 12: x=4.8572, y=4.1588, z=-0.1252, Loss=0.54201644
Gen 13: x=4.8435, y=4.0397, z=-0.1290, Loss=0.35262840
Gen 14: x=4.9339, y=4.2448, z=-0.1346, Loss=0.14472814
Gen 15: x=4.9217, y=4.2251, z=-0.1332, Loss=0.04522788
Gen 16: x=4.9170, y=4.2211, z=-0.1328, Loss=0.01404234
Gen 17: x=4.91

In [12]:

# Q3
NUMBER_OF_EQUATIONS = 4


def q3fitness(ind, funcs):
    x_val, y_val, z_val, t_val = ind
    total_error = 0
    try:
        for f in funcs:
            val = f(x_val, y_val, z_val, t_val)
            total_error += abs(val)
    except Exception:

        return float('inf')
    return total_error


def q3_create_functions(parsed_eqs):
    funcs = []
    for eq in parsed_eqs:
        f = lambdify((x, y, z, t), eq.lhs - eq.rhs, 'numpy')
        funcs.append(f)
    return funcs


input_equations = []

for i in range(NUMBER_OF_EQUATIONS):
    equation = input(f"Enter equation {i+1} (e.g., 2*x + 3*y - z * t = 10): ")
    input_equations.append(equation)


parsed_eqs = [parse_equation_string(eq) for eq in input_equations]
funcs = q3_create_functions(parsed_eqs)
answer = geneticAlgorithm(4, functions=funcs, fitNessFunction=q3fitness)

final_loss = q3fitness(answer, funcs)
print("\n✅ Best solution found:")
print(f"x = {answer[0]:.2f}")
print(f"y = {answer[1]:.2f}")
print(f"z = {answer[2]:.2f}")
print(f"t = {answer[3]:.2f}")
print(f"🔍 Total error: {final_loss:.2f}")

Gen 1: x=-1688507.5349, y=586600.7501, z=-890676.1119, t=1049608.6429 Loss=6511454.11445559
Gen 2: x=318169.1828, y=291713.6537, z=142125.2039, t=-211608.7309 Loss=2112964.27617643
Gen 3: x=-89484.5508, y=-93494.7837, z=-133098.7825, t=76695.7890 Loss=813661.03395361
Gen 4: x=54173.2055, y=30949.6101, z=40034.5289, t=-44103.8232 Loss=227253.66704818
Gen 5: x=8916.8714, y=3326.6007, z=4670.9883, t=-10068.6269 Loss=50568.60776824
Gen 6: x=1707.9127, y=-551.2853, z=259.9870, t=-1326.4601 Loss=11460.46439393
Gen 7: x=-229.8756, y=-163.9494, z=-497.6496, t=-119.8226 Loss=4539.48274658
Gen 8: x=72.7758, y=84.8488, z=48.2150, t=-152.8236 Loss=1076.13479506
Gen 9: x=-63.0722, y=23.0624, z=-6.7812, t=29.2097 Loss=404.41981795
Gen 10: x=1.9396, y=5.2257, z=-3.8052, t=3.5785 Loss=76.36564230
Gen 11: x=14.6710, y=3.3852, z=8.6194, t=-8.3933 Loss=26.02266671
Gen 12: x=11.6668, y=4.2584, z=6.7636, t=-4.9324 Loss=10.55970516
Gen 13: x=7.8889, y=2.5148, z=4.8817, t=-2.2450 Loss=3.02368961
Gen 14: x=7.